Put merged openai batch call output jsonl files in `./mmsci-exps/eval/output/openai_output_w_settings_merged/`.

The directory structure would be like:
- ./mmsci-exps/eval/output/openai_output_w_settings_merged/
  - gpt-4-turbo_wo_abstract_wo_content_image_caption_generation_data.jsonl
  - gpt-4-turbo_wo_abstract_w_content_image_caption_generation_data.jsonl
  - gpt-4-turbo_w_abstract_wo_content_image_caption_generation_data.jsonl
  - gpt-4-turbo_wo-cot_image_caption_matching_data.jsonl
  - gpt-4-turbo_w-cot_image_caption_matching_data.jsonl
  - gpt-4o_wo_abstract_wo_content_image_caption_generation_data.jsonl
  - gpt-4o_wo_abstract_w_content_image_caption_generation_data.jsonl
  - gpt-4o_w_abstract_wo_content_image_caption_generation_data.jsonl
  - gpt-4o_wo-cot_image_caption_matching_data.jsonl
  - gpt-4o_w-cot_image_caption_matching_data.jsonl

In [10]:
# reformat openai captioning results
import os
import json
import glob

base_input_dir = './output/openai_output_w_settings_merged'
k = 3
task = 'generation'
output_dir = f'./output/image_caption_{task}'

base_data_dir = '../../mmsci-data/benchmark/dev/'
input_data = json.load(open(os.path.join(base_data_dir, f'image_caption_{task}_data.json')))
input_data_mapping = {i: item for i, item in enumerate(input_data)}

def reformat_caption_generation(input_filepath, output_filepath):
    output_list = []
    with open(input_filepath, 'r') as fin:
        for line in fin.readlines():
            info = json.loads(line)
            key = int(info['custom_id'])
            answers = []
            for ans_info in info['response']['body']["choices"]:
                answers.append(ans_info['message']['content'])
            info = input_data_mapping[key]
            info['prediction'] = answers
            output_list.append(info)
    with open(output_filepath, 'w') as fout:
        json.dump(output_list, fout, indent=4)

In [12]:
file_list = glob.glob(os.path.join(base_input_dir, f'*{task}*'))

for filepath in file_list:
    filename = filepath.split('/')[-1]
    print(filename)
    model_name = filename.split('_')[0]
    w_abs = filename.find('w_abstract') > -1
    w_content = filename.find('w_content') > -1
    tag = f'abs{str(w_abs)}_ctx{str(w_content)}'
    cur_output_dir = os.path.join(output_dir, tag, f'k_{k}')
    os.makedirs(cur_output_dir, exist_ok=True)
    reformat_caption_generation(
        input_filepath=filepath,
        output_filepath=os.path.join(cur_output_dir, f'{model_name}.json')
    )

gpt-4-turbo_wo_abstract_wo_content_image_caption_generation_data.jsonl
gpt-4-turbo_wo_abstract_w_content_image_caption_generation_data.jsonl
gpt-4o_wo_abstract_wo_content_image_caption_generation_data.jsonl
gpt-4o_w_abstract_wo_content_image_caption_generation_data.jsonl
gpt-4-turbo_w_abstract_wo_content_image_caption_generation_data.jsonl
gpt-4o_wo_abstract_w_content_image_caption_generation_data.jsonl


In [15]:
# reformat matching results
import os
import json
import glob
from collections import defaultdict

base_input_dir = './output/openai_output_w_settings_merged'
k = 5
task = 'matching'
output_dir = f'./output/image_caption_{task}'

base_data_dir = '../../mmsci-data/benchmark/dev/'
all_input_data = json.load(open(os.path.join(base_data_dir, f'image_caption_{task}_data.json')))
input_data_mapping = defaultdict(dict)
for setting, input_data in enumerate(all_input_data):
    input_data_mapping[setting+1] = {i: item for i, item in enumerate(input_data)}
print(input_data_mapping.keys())

def reformat_caption_matching(input_filepath, model_name, tag):
    all_output_list = defaultdict(list)
    with open(input_filepath, 'r') as fin:
        for line in fin.readlines():
            info = json.loads(line)
            setting, key = info['custom_id'].split('_')  # f'{setting+1}_{str(idx)}'
            setting, key = int(setting), int(key)
            answers = []
            for ans_info in info['response']['body']["choices"]:
                answers.append({
                    'answer': ans_info['message']['content']
                })
            info = input_data_mapping[setting][key]
            info['prediction'] = answers
            all_output_list[setting].append(info)
    for setting, output_list in all_output_list.items():
        cur_output_dir = os.path.join(output_dir, tag, f'setting-{setting}', f'k_{k}')
        os.makedirs(cur_output_dir, exist_ok=True)
        output_filepath = os.path.join(cur_output_dir, f'{model_name}.json')
        with open(output_filepath, 'w') as fout:
            json.dump(output_list, fout, indent=4)

dict_keys([1, 2, 3])


In [16]:
file_list = glob.glob(os.path.join(base_input_dir, f'*{task}*'))

for filepath in file_list:
    filename = filepath.split('/')[-1]
    print(filename)
    model_name = filename.split('_')[0]
    tag = 'w_cot' if filename.find('w-cot') > -1 else 'wo_cot'
    print(tag, model_name)
    reformat_caption_matching(
        input_filepath=filepath, 
        model_name=model_name, 
        tag=tag,
    )

gpt-4-turbo_w-cot_image_caption_matching_data.jsonl
w_cot gpt-4-turbo
gpt-4-turbo_wo-cot_image_caption_matching_data.jsonl
wo_cot gpt-4-turbo
gpt-4o_wo-cot_image_caption_matching_data.jsonl
wo_cot gpt-4o
gpt-4o_w-cot_image_caption_matching_data.jsonl
w_cot gpt-4o
